In [1]:
import sklearn
import corus
import pandas as pd
import numpy as np
import nltk
import json
import tqdm

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import string
import re

from pprint import pprint

from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/partnadem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/partnadem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
MAX_LENTA_DATASET_ROWS = 10000
RANDOM_STATE = 5555
NUM_TOPICS = 10
CHUNK_SIZE = 750

#### Загрузить набор данных Lenta.ru с помощью пакета Corus

In [6]:
!pip install requests

     ---------------------------------------- 0.0/61.5 kB ? eta -:--:--
     ---------------------------------------- 61.5/61.5 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/158.3 kB ? eta -:--:--
   -------------------------------------- - 153.6/158.3 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 158.3/158.3 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---------------------------------------- 100.4/100.4 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/124.2 kB ? eta -:--:--
   ---------------------------------------- 124.2/124.2 kB 7.1 MB/s eta 0:00:00


In [8]:
import requests
from corus import load_lenta2

url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2"
response = requests.get(url)

open("lenta-ru-news.csv.bz2", "wb").write(response.content)

path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/hungarnn/',
    title='1914. Русские войска вступили в\xa0пределы Венгрии  ',
    text='Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии

In [4]:
wnl = WordNetLemmatizer()
STOP_WORDS = stopwords.words('russian')
external_stop_word_1 = open(
    "stopwords-ru.txt", "r", encoding="utf-8").read().replace('\n', ' ').split(" ")

raw_json = open(
    "stopwords-ru.json", "r", encoding="utf-8").read()
external_stop_word_2 = json.loads(raw_json)

external_stop_words = set(external_stop_word_1 + external_stop_word_2)

STOP_WORDS.extend(external_stop_words)


def lemmatize_token(token): return wnl.lemmatize(token)


def normalize_text(text):
    split_text = re.sub('\w*\d\w*', '', text).translate(str.maketrans(
        '', '', string.punctuation)).lower().split()
    result = ' '.join([lemmatize_token(word)
                      for word in split_text if word not in STOP_WORDS])

    return result

<>:19: DeprecationWarning: invalid escape sequence \w
<>:19: DeprecationWarning: invalid escape sequence \w
/tmp/ipykernel_1195/670348397.py:19: DeprecationWarning: invalid escape sequence \w
  split_text = re.sub('\w*\d\w*', '', text).translate(str.maketrans(


In [5]:
def load_lenta_to_list(path, max_number=None):
    records = load_lenta2(path)
    texts, titles, dates, original_topics, raw_content = [], [], [], [], []
    for i, record in enumerate(records):
        texts.append(normalize_text(record.text))
        titles.append(record.title)
        dates.append(record.date)
        original_topics.append(record.topic)
        raw_content.append(record.text)

        if not max_number is None:
            if i >= max_number-1:
                break
    result = {}
    result['topic'] = titles
    result['date'] = dates
    result['original_topics'] = original_topics
    result['content'] = texts
    result['raw_content'] = raw_content

    return result

In [6]:
lenta_list = load_lenta_to_list(path, max_number=MAX_LENTA_DATASET_ROWS)

lentaFrame = pd.DataFrame(lenta_list)
lentaFrame

,topic,date,original_topics,content,raw_content
0,1914. Русские войска вступили в пределы Венгрии,1914-09-16,Библиотека,бои сопоцкина друскеник закончились отступлени...,Бои у Сопоцкина и Друскеник закончились отступ...
1,1914. Празднование столетия М.Ю. Лермонтова от...,1914-09-16,Библиотека,министерство народного просвещения виду происх...,"Министерство народного просвещения, в виду про..."
2,1914. Das ist Nesteroff!,1914-09-17,Библиотека,штабскапитан п н нестеров днях увидев районе ж...,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,1914. Бульдог-гонец под Льежем,1914-09-17,Библиотека,фотографкорреспондент daily mirror рассказывае...,Фотограф-корреспондент Daily Mirror рассказыва...
4,1914. Под Люблином пойман швабский зверь,1914-09-18,Библиотека,лица приехавшие варшаву люблина передают доста...,"Лица, приехавшие в Варшаву из Люблина, передаю..."
...,...,...,...,...,...
9995,На границе Дагестана и Чечни будут стрелять в ...,2000-07-10,Россия,глава мвд дагестана приказал нарядам милиции п...,Глава МВД Дагестана приказал нарядам милиции н...
9996,Путин пересадит губернаторов в Госсовет,2000-07-10,Россия,представители кремля ведут активный диалог губ...,Представители Кремля ведут активный диалог с г...
9997,В Москву съедутся клоуны со всего мира,2000-07-10,Россия,москве июля открывается фестиваль планета клоу...,"В Москве 20 июля открывается фестиваль ""Планет..."
9998,Старшая Уильямс выигрывает чемпионский титул,2000-07-10,Спорт,чисто американским получился финал уимблдонско...,Чисто американским получился финал Уимблдонск...


#### Обучить LDA модель, постараться подобрать адекватные параметры (num_topics, passes, alpha, iterations…)

In [7]:
tokenized_content = lentaFrame.content.apply(lambda x: x.split())
id2word = corpora.Dictionary(tokenized_content)

In [8]:
corpus = [id2word.doc2bow(text) for text in tokenized_content]
corpus_example = [[(id2word[id], freq) for id, freq in cp]
                  for cp in corpus[:2]]
corpus_example[0][:50]

[('«русский', 1),
 ('австрийцев', 1),
 ('автомобили', 1),
 ('артиллерийском', 1),
 ('артиллерийскую', 1),
 ('артиллерию', 1),
 ('батареей', 1),
 ('бежала', 1),
 ('безуспешными', 1),
 ('ближе', 1),
 ('бои', 1),
 ('большая', 1),
 ('борьбу', 1),
 ('бою', 1),
 ('бросив', 1),
 ('венгрии', 1),
 ('взяли', 1),
 ('вступили', 1),
 ('вылазки', 1),
 ('высот', 1),
 ('галиции', 1),
 ('гарнизона', 1),
 ('германской', 1),
 ('германцев', 1),
 ('дембицу', 1),
 ('достиг', 1),
 ('друскеник', 1),
 ('закончились', 1),
 ('заняли', 1),
 ('захватываются', 1),
 ('значительного', 1),
 ('инвалид»', 1),
 ('калибры', 1),
 ('колонна', 1),
 ('крепости', 1),
 ('крепостью', 1),
 ('материальная', 1),
 ('напряжения', 1),
 ('начал', 1),
 ('нашей', 1),
 ('неприятель', 1),
 ('неприятельский', 1),
 ('новые', 1),
 ('обнаруживается', 1),
 ('обоз', 1),
 ('обстреливалась', 1),
 ('огонь', 1),
 ('орудия', 1),
 ('осовцу', 1),
 ('остаются', 1)]

In [9]:
def get_coherence(corpus, topics, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           random_state=RANDOM_STATE,
                                           chunksize=CHUNK_SIZE,
                                           num_topics=topics,
                                           alpha=a,
                                           eta=b,
                                           workers=3)

    coherence_model_lda = CoherenceModel(
        model=lda_model, texts=tokenized_content, dictionary=id2word, coherence='c_v')

    return coherence_model_lda.get_coherence()

In [10]:
%%time

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=NUM_TOPICS,
                                       random_state=RANDOM_STATE,
                                       chunksize=CHUNK_SIZE,
                                       alpha='symmetric',
                                       per_word_topics=True,
                                       workers=3)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"россии" + 0.004*"сообщает" + 0.004*"словам" + 0.003*"чечне" + '
  '0.003*"боевиков" + 0.003*"новости" + 0.003*"риа" + 0.002*"заявил" + '
  '0.002*"президента" + 0.002*"области"'),
 (1,
  '0.007*"россии" + 0.004*"заявил" + 0.004*"сообщает" + 0.004*"словам" + '
  '0.003*"сообщил" + 0.003*"президента" + 0.002*"рф" + 0.002*"чечни" + '
  '0.002*"новости" + 0.002*"президент"'),
 (2,
  '0.006*"россии" + 0.004*"новости" + 0.004*"риа" + 0.004*"сообщили" + '
  '0.003*"мвд" + 0.003*"сообщает" + 0.003*"боевиков" + 0.003*"районе" + '
  '0.003*"словам" + 0.003*"данным"'),
 (3,
  '0.003*"сообщает" + 0.003*"рф" + 0.003*"россии" + 0.003*"риа" + '
  '0.003*"новости" + 0.003*"рублей" + 0.003*"словам" + 0.003*"заявил" + '
  '0.002*"сша" + 0.002*"процента"'),
 (4,
  '0.005*"сообщает" + 0.003*"россии" + 0.003*"компании" + 0.003*"словам" + '
  '0.003*"заявил" + 0.003*"риа" + 0.002*"долларов" + 0.002*"новости" + '
  '0.002*"чемпионата" + 0.002*"передает"'),
 (5,
  '0.006*"россии" + 0.005*"риа" 

#### Визуализировать результаты работы LDA с помощью pyLDAvis

In [11]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.014027 -0.034678       1        1  19.623597
5     -0.031853 -0.018876       2        1  13.977175
7      0.012713 -0.021740       3        1  12.666417
2     -0.074308  0.015758       4        1  10.085041
9      0.007449 -0.018670       5        1   9.982824
6      0.022601  0.002488       6        1   8.736820
4      0.029626  0.052366       7        1   8.051455
1      0.006399 -0.004778       8        1   7.066069
3      0.032139  0.008308       9        1   6.313292
0     -0.018793  0.019821      10        1   3.497310, topic_info=               Term         Freq        Total Category  logprob  loglift
471          россии  5233.000000  5233.000000  Default  30.0000  30.0000
1369       боевиков  1106.000000  1106.000000  Default  29.0000  29.0000
1506          чечне  1339.000000  1339.000000  Default  28.0000  28.0000
4548         рублей  1156.000000  1156.000000  Default  27.0000  27.0000
998        сообщили  1387.000000  1387.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
2061            сша    51.186226  2290.785658  Topic10  -6.5378  -0.4480
521          данным    46.115151  1607.864655  Topic10  -6.6421  -0.1983
608        долларов    48.997011  2068.004266  Topic10  -6.5815  -0.3894
1709       компании    41.959318  1339.302010  Topic10  -6.7366  -0.1100
652   правительства    41.324103  1228.190613  Topic10  -6.7518  -0.0387

[898 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
7127       6  0.933643      atlantis
29203      5  0.948461            bt
61450      1  0.046649          card
61450      2  0.746382          card
61450      3  0.093298          card
...      ...       ...           ...
36101      6  0.028065  ястржембский
36101      7  0.028065  ястржембский
36101      8  0.196455  ястржембский
36101      9  0.022452  ястржембский
36101     10  0.145938  ястржембский

[3168 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 8, 3, 10, 7, 5, 2, 4, 1])

#### Посчитать внутренние метрики обученных моделей LDA (с разными параметрами) и сравнить, соответствует ли метрика визуальному качеству работы моделей

In [12]:
get_coherence(corpus, NUM_TOPICS, 1, 1)

0.360842956224781

In [14]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 3
max_topics = 12
step_size = 3
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),
               corpus]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                 }

if 1 == 0:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha) *
                     len(topics_range)))

    for k in topics_range:
        for a in alpha:
            for b in beta:
                cv = get_coherence(corpus=corpus,
                                   topics=k, a=a, b=b)

                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)

                pbar.update(1)

    pd.DataFrame(model_results).to_csv(
        './lda_tuning_results.csv', index=False)
    pbar.close()

#### Обучить модель BigARTM, использовать не менее двух регуляризаторов, оценить качество с помощью метрик

In [ ]:
!pip uninstall -y protobuf bigartm bigartm10
!pip install bigartm
!pip install protobuf==3.20

Found existing installation: protobuf 4.24.4
Uninstalling protobuf-4.24.4:
  Successfully uninstalled protobuf-4.24.4
Found existing installation: bigartm 0.9.2
Uninstalling bigartm-0.9.2:
  Successfully uninstalled bigartm-0.9.2
  Using cached bigartm-0.9.2-cp39-cp39-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (2.1 MB)
  Using cached protobuf-4.24.4-cp37-abi3-manylinux2014_x86_64.whl.metadata (540 bytes)
Using cached protobuf-4.24.4-cp37-abi3-manylinux2014_x86_64.whl (311 kB)
  Using cached protobuf-3.20.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4


In [15]:
import artm
from sklearn.feature_extraction.text import CountVectorizer
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [16]:
tokenized_content = lentaFrame.content.to_list()

cv = CountVectorizer(max_features=1000, max_df=0.8,
                     min_df=2, ngram_range=(1, 2))
n_wd = np.array(cv.fit_transform(tokenized_content).todense()).T

n_wd.shape

(1000, 10000)

In [17]:
token_list = [i for i in cv.vocabulary_.keys()]
bv = artm.BatchVectorizer(data_format='bow_n_wd',
                          n_wd=n_wd, vocabulary=token_list)

/home/partnadem/.pyenv/versions/3.9.12/envs/wsl-3.9.12/lib/python3.9/site-packages/artm/batches_utils.py:227: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
